# Taproot: MuSig2 + Tapscript Spends

In this module we'll be making a series of MuSig2 scripts.

We'll then build a MAST of the scripts, and lock up some bitcoin to them. 
Finally we'll practice unlocking the script using a tapscript spend.

## Picking our private keys

We'll need three private keys. Pick three keys (integers) that are greater than 1 and less than `n`,

`n = 115792089237316195423570985008687907852837564279074904382605163141518161494337`

In [ ]:
sk1 = 

In [1]:
!mkdir class-node

In [2]:
!bitcoind -regtest -datadir=$(pwd)/class-node -fallbackfee=0.00000012 -daemon 

Bitcoin Core starting


In [5]:
!alias bcli="bitcoin-cli -regtest -datadir=$(pwd)/class-node"

In [8]:
!bitcoin-cli -regtest -datadir="$(pwd)/class-node" createwallet base58

error: Could not locate RPC credentials. No authentication cookie could be found, and RPC password is not set.  See -rpcpassword and -stdinrpcpass.  Configuration file: (/home/niftynei/dev/taproots/jupyter/class-node/bitcoin.conf)
